In [1]:
25*6/10

15.0

In [2]:
from gmshtools.CircRect6 import CircRect


In [3]:
import gmsh 
import numpy as np

gmsh.initialize()

gmsh.model.add("t1")

# cilindro
Ly = 25
Lx = 60
tn  = 2.934
tl = 0.475
r  = 3*tl/8
d_Y  = 0.74297
d_X  = 0.55872
A = np.pi*r**2



R = 10/6
nx_fibres = 5


z0 = tn/2
dx = 0.1 
dz = 0.1

hx = R
hz = tl
Lz = 2*2*hz + 2*z0 

def CreateCylinder_Lxx(x0,z0):

    ellipse_1 = CircRect(x0, 0, z0, 
                         r, d_X, 
                         [0,1,0] ,            # vec_z 
                         [1,0,0])             # vec_x
    
    ellipse_2 = CircRect(x0, +Ly/2, z0,
                            r, d_X, 
                            [0,1,0] ,            # vec_z 
                            [1,0,0])             # vec_x
    gmsh.model.occ.synchronize()

    thrusection = gmsh.model.occ.addThruSections([ellipse_1, ellipse_2])

    gmsh.model.occ.synchronize()

    return thrusection[0][1]

def CreateCylinder_Lyy(y0,z0):
    
    ellipse_1 = CircRect(0, y0, z0,
                            r, d_Y, 
                            [1,0,0] ,            # vec_z 
                            [0,1,0])             # vec_x
    
    ellipse_2 = CircRect(nx_fibres*hx, y0, z0,
                            r, d_Y, 
                            [1,0,0] ,            # vec_z 
                            [0,1,0])             # vec_x
    
    gmsh.model.occ.synchronize()



    thrusection = gmsh.model.occ.addThruSections([ellipse_1, ellipse_2])

    gmsh.model.occ.synchronize()

    return thrusection[0][1]

FP1_list = []
FM1_list = []
for ix in range(nx_fibres):
    FibrePlus_01  = CreateCylinder_Lxx(hx/2 + hx*ix, +z0 + 3*hz/2 )
    FibreMinus_01 = CreateCylinder_Lxx(hx/2 + hx*ix, -z0 - 3*hz/2 )

    FP1_list.append(FibrePlus_01)
    FM1_list.append(FibreMinus_01)



gmsh.model.occ.synchronize()
# 
#

nfilaments = int(0.5*25*6/10)

yspan = np.arange(0, Ly/2 - hx, hx)
yspan = yspan + hx/2

FP2_list = []
FM2_list = []
for iy in yspan:
    FibrePlus_02  = CreateCylinder_Lyy(iy, +z0 + hz/2 )
    FibreMinus_02 = CreateCylinder_Lyy(iy, -z0 - hz/2 )

    FP2_list.append(FibrePlus_02)
    FM2_list.append(FibreMinus_02)

FP2_list.extend(FP1_list)
FM2_list.extend(FM1_list)

# add Physical Groups
fibres_tags = np.concatenate([FP1_list, FM1_list, FP2_list, FM2_list])
f1 = gmsh.model.addPhysicalGroup(3, fibres_tags)
gmsh.model.setPhysicalName(3, f1, "Fibres")


gmsh.model.occ.synchronize()
box_01 = gmsh.model.occ.addBox(0, 0 , + z0 ,
                               nx_fibres*hx   ,  Ly/2   ,  2*hz    )

box_02 = gmsh.model.occ.addBox(0, 0 , - z0 ,
                               nx_fibres*hx   ,  Ly/2   ,  -2*hz    )

gmsh.model.occ.synchronize()

box_01_cut = gmsh.model.occ.cut([(3, box_01)], 
                   [ (3,i) for i in FP2_list],
                   removeTool=False)

box_02_cut = gmsh.model.occ.cut([(3, box_02)], 
                   [(3,i) for i in FM2_list],
                   removeTool=False)




gmsh.model.occ.synchronize()



In [4]:



# # box 
# #
# gmsh.model.occ.synchronize()
# #

box_big_mid = gmsh.model.occ.addBox(0      , 0     , -z0,
                                    Lx/2   ,  Ly/2   ,  2*z0    )


box_sup = gmsh.model.occ.addBox(0       ,   0     , z0 + hz,
                                Lx/2    ,  Ly/2   ,  hz       )

box_inf = gmsh.model.occ.addBox(0       ,   0     , -z0 - hz,
                                Lx/2    ,  Ly/2   ,  -hz       )

gmsh.model.occ.synchronize()
volumes = gmsh.model.getEntities(3)



box_big = gmsh.model.occ.addBox(0     ,  0    , -Lz/2,
                                Lx/2  ,  Ly/2   ,  Lz    )

gmsh.model.occ.synchronize()

# cut box_big - volumes
box_big = gmsh.model.occ.fragment([(3,box_big)], volumes)



gmsh.model.occ.synchronize()



# buscar la caja que tiene como media z= 0


In [5]:
volumes = gmsh.model.getEntities(3)
com = [ gmsh.model.occ.getCenterOfMass(3, i[1]) 
       for i in volumes ]


sort_com_index = np.argsort(np.abs(np.array(com)[:,2]))

In [6]:
id = volumes[sort_com_index[0]][1]

In [7]:
# physical group
f2 = gmsh.model.addPhysicalGroup(3, [id])
gmsh.model.setPhysicalName(3, f2, "Nucleo")
# 
gmsh.model.occ.synchronize()

#
resina = [ i[1] for i in volumes if i[1] != id ]
resina = [ i for i in resina if i not in fibres_tags]


# sort by x
com = [ gmsh.model.occ.getCenterOfMass(3, i) 
       for i in resina ]

sort_com_index = np.argsort(np.array(com)[:,0])

resina = [resina[i] for i in sort_com_index]


In [8]:
resina

[34, 29, 32, 28, 35, 30, 31, 33]

In [9]:
resina[:4]

[34, 29, 32, 28]

In [10]:

f3 = gmsh.model.addPhysicalGroup(3, resina[:4])
gmsh.model.setPhysicalName(3, f3, "Resina")

capas_hom = resina[4:]
# sort 
com = [ gmsh.model.occ.getCenterOfMass(3, i) for i in capas_hom ]
# sort by z
sort_com_index = np.argsort(np.array(com)[:,2])

capas_hom = [capas_hom[i] for i in sort_com_index]

f4 = gmsh.model.addPhysicalGroup(3, [capas_hom[0],capas_hom[3]])
gmsh.model.setPhysicalName(3, f4, "CAPAS_2")

f5 = gmsh.model.addPhysicalGroup(3, [capas_hom[1],capas_hom[2]])
gmsh.model.setPhysicalName(3, f5, "CAPAS_1")


#

In [11]:
def BoxRefine(x1,x2,y1,y2,z1,z2,vin,vout,Thickness=0.25):
    

    field_tag = gmsh.model.mesh.field.add("Box")

    gmsh.model.mesh.field.setNumber(field_tag, "VIn", vin)
    gmsh.model.mesh.field.setNumber(field_tag, "VOut", vout)

    gmsh.model.mesh.field.setNumber(field_tag, "XMin", x1)
    gmsh.model.mesh.field.setNumber(field_tag, "XMax", x2)

    gmsh.model.mesh.field.setNumber(field_tag, "YMin", y1)
    gmsh.model.mesh.field.setNumber(field_tag, "YMax", y2)

    gmsh.model.mesh.field.setNumber(field_tag, "ZMin", z1)
    gmsh.model.mesh.field.setNumber(field_tag, "ZMax", z2)

    # Thickness
    gmsh.model.mesh.field.setNumber(field_tag, "Thickness", Thickness)

    return field_tag

In [12]:
#field
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 9) # 20

mesh_inter = 1.5*r
mesh_min = 2.0*r 
mesh_max = 7.0*r
mesh_out = 10.0*r

dx_per = 1.05
#  
field_base = BoxRefine(0    ,  Lx/2, 
                       0    ,  Ly/2, 
                      -Lz/2 ,  Lz/2,
                    mesh_out, mesh_out)

field_mid = BoxRefine(0    ,   dx_per*nx_fibres*hx,
                          0    ,  Ly/2, 
                         -Lz/2 ,  Lz/2,
                      mesh_max, mesh_out)
# box_01
field_box_01 = BoxRefine(0       , dx_per*nx_fibres*hx,
                         0       , Ly/2, 
                         z0 , 2*hz + z0,
                         mesh_min, mesh_out,Thickness=0.25)

field_box_02 = BoxRefine(0       , dx_per*nx_fibres*hx,
                            0       , Ly/2, 
                            -z0 - 2*hz, -z0,
                            mesh_min, mesh_out,Thickness=0.25)


list_field = [field_base, field_box_01, field_box_02, field_mid]

field_min = gmsh.model.mesh.field.add("Min")
gmsh.model.mesh.field.setNumbers(field_min, "FieldsList", list_field)

gmsh.model.mesh.field.setAsBackgroundMesh(field_min)


In [13]:



gmsh.model.mesh.generate(3)

# optimization netgen
gmsh.model.mesh.optimize("Netgen")

# second order 
gmsh.model.mesh.setOrder(2)
gmsh.write("t1.inp")

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Circle)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Circle)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Circle)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Circle)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Bezier)
Info    : [ 10%] Meshing curve 14 (Bezier)


Info    : [ 10%] Meshing curve 15 (BSpline)
Info    : [ 10%] Meshing curve 16 (BSpline)
Info    : [ 10%] Meshing curve 17 (Bezier)
Info    : [ 10%] Meshing curve 18 (BSpline)
Info    : [ 10%] Meshing curve 19 (Circle)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Circle)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 24 (Line)
Info    : [ 10%] Meshing curve 25 (Circle)
Info    : [ 10%] Meshing curve 26 (Line)
Info    : [ 10%] Meshing curve 27 (Line)
Info    : [ 10%] Meshing curve 28 (Circle)
Info    : [ 10%] Meshing curve 29 (Line)
Info    : [ 10%] Meshing curve 30 (Line)
Info    : [ 10%] Meshing curve 31 (Bezier)
Info    : [ 10%] Meshing curve 32 (Bezier)
Info    : [ 10%] Meshing curve 33 (BSpline)
Info    : [ 10%] Meshing curve 34 (BSpline)
Info    : [ 10%] Meshing curve 35 (Bezier)
Info    : [ 10%] Meshing curve 36 (BSpline)
Info    : [ 10%] Meshing curve 37 (Circle)
Info    : [ 10%] Mesh

Info    : Done meshing order 2 (Wall 2.33445s, CPU 2.34266s)
Info    : Writing 't1.inp'...
Info    : Done writing 't1.inp'


In [14]:
import pandas as pd
path = "../../../01_metodo/02_flexion/04_optim_stress/df_pred.csv"
# path = r"C:\Users\djoroya\Documents\GitHub\CITISENS_Composite_v2\scripts_finals\01_metodo\02_flexion\04_optim_stress\df_pred.csv"
df_pred = pd.read_csv(path, sep=",")
df_pred

# path = r"C:\Users\djoroya\Documents\GitHub\CITISENS_Composite_v2\scripts_finals\01_metodo\02_flexion\01_optim\mat.csv"
path = "../../../01_metodo/02_flexion/01_optim/mat.csv"
df_mat = pd.read_csv(path, sep=",")
df_mat

# df_layers = r"C:\Users\djoroya\Documents\GitHub\CITISENS_Composite_v2\scripts_finals\01_metodo\02_flexion\01_optim\layers.csv"
df_layers = "../../../01_metodo/02_flexion/01_optim/layers.csv"
df_layers = pd.read_csv(df_layers, sep=",")
df_layers

,Unnamed: 0,Evec [GPa]
0,X,10.156516
1,SX,8.410848
2,Y,21.993825
3,SY,8.379984
4,Nucleo,0.880000


In [15]:
E1 = 1e3*df_layers.loc[2].values[1]  # Y
E2 = 1e3*df_layers.loc[0].values[1]  # X

Ec = 1e3*df_mat.loc[1].values[1] 

nu = 0.3 
Em = 1e3*df_mat.loc[0].values[1] 
En = 880


In [16]:
from djccx.inp.inp import  inp 

inp_f = inp("t1.inp")

inp_f.remove_by_type(2)

nodes = inp_f.nodes.df

nodes_sym = nodes.copy()
nodes_sym = nodes_sym[ nodes_sym["x"] < 1e-3] 

nset_sym_x = inp_f.CreateNsetFromIds(nodes_sym.index, "sym_x")

nodes_sym = nodes.copy()
nodes_sym = nodes_sym[ nodes_sym["y"] < 1e-3] 

nset_sym_y = inp_f.CreateNsetFromIds(nodes_sym.index, "sym_y")

nodes_fixed = nodes.copy()
nodes_fixed = nodes_fixed[ nodes_fixed["x"] > 0.5*Lx  - 1e-3]
nodes_fixed = nodes_fixed[ nodes_fixed["z"] < -0.5*Lz + 1e-3]

nset_fixed = inp_f.CreateNsetFromIds(nodes_fixed.index, "fixed")

nodes_force = nodes.copy()
nodes_force = nodes_force[ nodes_force["x"] < 1e-3]
nodes_force = nodes_force[ nodes_force["z"] > 0.5*Lz - 1e-3]

nset_force = inp_f.CreateNsetFromIds(nodes_force.index, "force")

# materials 
mat_fibras = inp_f.CreateElasticMaterial("FIBRES", Ec, nu)
mat_resina = inp_f.CreateElasticMaterial("RESINA", Em, nu)
mat_capas_1 = inp_f.CreateElasticMaterial("CAPAS_1", E1, nu)
mat_capas_2 = inp_f.CreateElasticMaterial("CAPAS_2", E2, nu)
mat_nucleo = inp_f.CreateElasticMaterial("NUCLEO", En, nu)


# elsets 
fibres_elset = inp_f.select("FIBRES","elset")
resina_elset = inp_f.select("RESINA","elset")
capas_1_elset = inp_f.select("CAPAS_1","elset")
capas_2_elset = inp_f.select("CAPAS_2","elset")
nucleo_elset = inp_f.select("NUCLEO","elset")

inp_f.CreateSolidSection( fibres_elset, mat_fibras)
inp_f.CreateSolidSection( resina_elset, mat_resina)
inp_f.CreateSolidSection( capas_1_elset, mat_capas_1)
inp_f.CreateSolidSection( capas_2_elset, mat_capas_2)
inp_f.CreateSolidSection( nucleo_elset, mat_nucleo)
#

istep = inp_f.CreateStaticStep()
istep.CreateBoundary(nset_fixed,3,0)
istep.CreateBoundary(nset_sym_x,1,0)
istep.CreateBoundary(nset_sym_y,2,0)

disp = 5
istep.CreateBoundary(nset_force, 3, -disp)




Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified
Authorization required, but no authorization protocol specified


Wine está instalado. Versión: wine-9.0
Wine is installed. Calculix will be run with windows binary


*BOUNDARY
FORCE, 3, -5

In [17]:
fibres_elset = inp_f.select("FIBRES","elset")


In [18]:
inp_f.run("output",mkdirforce=True)

Running Calculix at:  output
Output file:  /home/djoroya/projects/CITISENS/CITISENS_Composite/scripts_finals/02_metodo/02_flexion/TPF2/output/out.txt
Command:
 wine /home/djoroya/projects/CITISENS/CITISENS_Composite/src/djccx/bin/ccx_dynamic.exe main > out.txt
Error reading cvf file

pid:  894683 

Calculix finished



{'data':           node          x             y        z        D1        D2  \
 node                                                                   
 1            1   0.553972  0.000000e+00  2.00137 -0.019250  0.000000   
 2            2   0.553972  0.000000e+00  2.35763 -0.034742  0.000000   
 3            3   0.833333  1.000000e-08  2.00137 -0.021726  0.000000   
 4            4   1.112690  0.000000e+00  2.00137 -0.024058  0.000000   
 5            5   1.112690  0.000000e+00  2.35763 -0.040539  0.000000   
 ...        ...        ...           ...      ...       ...       ...   
 453190  453190   8.544090  4.341660e+00  2.35991 -0.235772  0.018987   
 453191  453191   8.411230  8.099390e+00  2.15743 -0.208189  0.034144   
 453192  453192   8.555750  1.562980e+00  2.36035 -0.236889  0.006732   
 453193  453193   8.390300  1.227170e+01  2.06522 -0.208855  0.053043   
 453194  453194  28.965100  8.990480e+00  2.05997 -0.414576  0.010802   
 
               D3        SXX       SYY   

In [19]:
0.5*Lz - 1e-3

2.416